In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers

In [75]:
# Get List of Every Result and Save
make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/HTML/"+ make + model +".html", 'w').close()
ids, urls = helpers.get_listings2(make)
data = ''
listings = []
urlend = '&results=items'
pgend = 15
delimiter = "%newline%"

# Pull HTML data from each url, defined here so that it can run with joblib.Parallel
def get_data(url):
    r = requests.get(url, allow_redirects=False)
    soup = BeautifulSoup(r.content, "html.parser")
    return str(soup)

# Iterate through BaT specific Id's and show more pages for each listing
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    tempurl = helpers.get_urls(urlstart, urlend, pgend, urls)
    listings = tempurl
    listings = listings[len(ids):len(listings)]
    souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in listings)

    temp = delimiter.join(souplist)
    data += delimiter + temp 

# Write data to file 
with open("./data/HTML/"+ make + model +".html", "a") as file:
    file.write(data)

display('done')


83


In [ ]:
carlist = []
Car = []
### Read Data and Save to DataFrame ###

with open("./data/HTML/"+ make + model +".html", "r") as file:
    test_data = file.read()

out = test_data.split(delimiter)

#Iterate through each listings and test again to filter out non-car listings using title this time
for i in range(0,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if (
        "Parts" not in title 
        and "Tool" not in title 
        and "Memorabilia" not in title 
        and "Luggage" not in title 
        and "Tool Kit" not in title 
        and "Removable" not in title 
        and "Gearbox" not in title 
        and title != 'None' 
        and 'listing' in listings[i]
    ):
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.get_desc(listings[i])

        #Get Year
        year = desc[0:4]
        
        #Check if Sold and get highest bid
        sold, price = helpers.get_sale_price(title, loc)

        #Get Transmission
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location
        town, state = helpers.get_location(loc)
        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        try:
            index = title.index('Lot #')
            lotnumb = title[index+5:index+12].partition(')')[0]
        except:
            lotnumb = "N/A"

        #Check for Replica
        if "replica" in title or "Replica" in title:
            replica = 1
        else:
            replica = 0

        #Get Month
        month = helpers.get_month(title)
        
        #Get Year
        if '(' in title:
            index = title.index('(')
            yearsold = title[index-5:index-1]
        else:
            index = title.index('20')
            yearsold = title[index:index+5]
        
        #Get Engine and Number Matching
        numbmatch = 0
        if 'Numbers-Matching' in transmission:
            numbmatch = 1
        engine = helpers.get_engine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive, turbo, super = helpers.get_indicators(contents)
            
        #Get Mileage
        miles, milestmu = helpers.get_mileage(transmission)

        #Get Descriptions of options and chassis #
        chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, engine = helpers.get_engine_desc(essentials, engine)

        #Check Trans with Description
        if 'Manual' in transdesc:
            trans = 'Manual'
        elif 'Automatic' in transdesc:
            trans = 'Automatic'

        #Check Mileage with Description (TODO: remove try catch)
        try:
            if "K" in str(miles):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    milestmu = re.sub(r"[^0-9]", "", tempmile)
                    milestmu = int(milestmu) * 1000
            elif "K" in str(milestmu):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    miles = re.sub(r"[^0-9]", "", tempmile)
                    miles = int(miles) * 1000
        except:
            print('failed')
            
        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, \
            milestmu, engine, turbo, super, numbmatch, \
            hardtop, overdrive, scratch, paintbub, metalrepair, \
            replica, rust, refurbished, restored, project, sold, \
            month, yearsold, price, town, state, zagato, bertone, \
            chassis, specialdesc, mileagedesc, enginedesc, transdesc, \
            paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, listings[i], make, model]
        
        carlist.append(Car)
    
    else:
        print('Discarded')

# Export to DataFrame
df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)',\
    'Engine', 'Turbocharged', 'Supercharged', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', \
    'Replica', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', \
    'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description',\
    'Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description', 'Suspension Description', 'URL', 'Make', 'Model'])

display(df)


In [6]:
# Get List of Every Result and Save
make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/HTML/"+ make + model +".html", 'w').close()
ids, urls = helpers.get_listings(make, model)

In [103]:
def get_listings_no_model(make: str):
    url = "https://bringatrailer.com/" + make + "/"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    url = str(soup)
    i = 0
    urls = []
    while i < len(url):
                if url[i : i + 5] == '"url"':
                    tempurl = url[i + 7 : i + 200].partition('"')[0]
                    if (
                        "listing" in tempurl
                        and "parts" not in tempurl
                        and "hardtop" not in tempurl
                        and "tool" not in tempurl
                        and "memorabilia" not in tempurl
                        and "tool kit" not in tempurl
                        and "inline" not in tempurl
                        and "removable" not in tempurl
                        and "gearbox" not in tempurl
                    ):
                        if "\\" in tempurl:
                            tempurl = tempurl.replace("\\", "")
                            if tempurl not in urls:
                                urls.append(tempurl.replace("\\", ""))
                i += 1

    if len(urls) > 30:
        urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?s=' + make + '&sort=td&page='
        urlend = '&results=items'
        url = []
        pgnum = 1
        pgend = 10

        # Iterate through each page (Show More in BaT) and collect urls
        while pgnum < pgend:
            url.append(urlstart + str(pgnum) + urlend)
            pgnum += 1

        # For each URL get soup
        for j in range(0, len(url)):
            r = requests.get(url[j])
            soup = BeautifulSoup(r.content, "html.parser")
            urltemp = str(soup)
            i = 0
            # Search page for all listings and try to filter out non-car listings
            while i < len(urltemp):
                if urltemp[i : i + 5] == '"url"':
                    tempurl = urltemp[i + 7 : i + 200].partition('"')[0]
                    if (
                        "listing" in tempurl
                        and "parts" not in tempurl
                        and "hardtop" not in tempurl
                        and "tool" not in tempurl
                        and "memorabilia" not in tempurl
                        and "tool kit" not in tempurl
                        and "inline" not in tempurl
                        and "removable" not in tempurl
                        and "gearbox" not in tempurl
                    ):
                        if "\\" in tempurl:
                            tempurl = tempurl.replace("\\", "")
                            if tempurl not in urls:
                                urls.append(tempurl.replace("\\", ""))
                i += 1
    return urls

In [105]:
urls = helpers.get_listings_no_model("Factory Five")
urls

['https://bringatrailer.com/listing/1965-factory-five-racing-mk4-shelby-cobra-52/',
 'https://bringatrailer.com/listing/factory-five-racing-mk3-roadster-4/',
 'https://bringatrailer.com/listing/ffr-roadster/',
 'https://bringatrailer.com/listing/2021-factory-five-type-65-daytona-coupe-2/',
 'https://bringatrailer.com/listing/1965-factory-five-racing-mkii-i-believe/',
 'https://bringatrailer.com/listing/1965-ford-factory-five-mkiv-cobra/',
 'https://bringatrailer.com/listing/factory-five-racing-mk2-roadster-4/',
 'https://bringatrailer.com/listing/2018-factory-five-racing-roadster/',
 'https://bringatrailer.com/listing/factory-five-racing-818c-2/',
 'https://bringatrailer.com/listing/2017-factory-five-racing-mk4/',
 'https://bringatrailer.com/listing/factory-five-racing-mk2-roadster-5/',
 'https://bringatrailer.com/listing/2004-factory-five-cobra-5/',
 'https://bringatrailer.com/listing/2013-factory-five-gtm-4/',
 'https://bringatrailer.com/listing/factory-five-mk-3-roadster/',
 'https:

In [ ]:
https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?s=Lancia&sort=td&page=2&results=items
https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=12732604&sort=td&page=2&results=items
https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?s=Lamborghini&sort=td&page=2&results=items